# **DELVE-BOSS matching and comparison**

In [1]:
import numpy as np
import pandas as pd
import pylab as plt
import h5py
import matplotlib
%matplotlib inline
from astropy.utils.data import download_file  #import file from URL
from matplotlib.ticker import NullFormatter
import astropy.table
import subprocess as sp
import glob
import healpy as hp

In [2]:
flist = glob.glob('/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat*.fits')
pxnums = []
for fname in flist:
    pxnums.append(int(fname.split('.')[1].split('_')[-1]))
pxnums = np.array(pxnums)
ra, dec = hp.pix2ang(32, pxnums, lonlat=True)

In [3]:
data_boss_north = astropy.table.Table.read('/data/des90.a/data/raulteix/data/speczs/galaxy_DR12v5_CMASSLOWZTOT_North.fits.gz')
data_boss_south = astropy.table.Table.read('/data/des90.a/data/raulteix/data/speczs/galaxy_DR12v5_CMASSLOWZTOT_South.fits.gz')
data_boss_all = astropy.table.vstack([data_boss_north, data_boss_south])
pxnums_boss = hp.ang2pix(32, data_boss_all['RA'], data_boss_all['DEC'], lonlat=True)
overlap_boss_delve = np.in1d(pxnums_boss, pxnums)

In [6]:
data_boss_all.columns
data_boss_all

RA,DEC,RUN,RERUN,CAMCOL,FIELD,ID,ICHUNK,IPOLY,ISECT,FRACPSF [5],EXPFLUX [5],DEVFLUX [5],PSFFLUX [5],MODELFLUX [5],FIBER2FLUX [5],R_DEV [5],EXTINCTION [5],PSF_FWHM [5],AIRMASS,SKYFLUX [5],EB_MINUS_V,IMAGE_DEPTH [5],IMATCH,Z,WEIGHT_FKP,WEIGHT_CP,WEIGHT_NOZ,WEIGHT_STAR,WEIGHT_SEEING,WEIGHT_SYSTOT,NZ,COMP,PLATE,FIBERID,MJD,FINALN,TILE [3],SPECTILE,ICOLLIDED,INGROUP,MULTGROUP
float64,float64,int32,bytes12,int32,int32,int32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int16,int32,int32,int32,int32
195.07128497875797,-0.5907713163844767,745,301,2,242,263,7,31442,12561,1.0 .. 1.0,5.3609266 .. 21.029993,7.2608414 .. 30.36549,-0.14064825 .. 11.323671,0.22083879 .. 25.720694,-0.018343575 .. 6.252284,74.92741 .. 5.007526,0.114992276 .. 0.032991964,1.5613878 .. 1.2457927,1.1966538,1.4503974 .. 34.529015,0.02230694,1.2488135 .. 5.2008977,1,0.4656479,0.22490303,1.0,1.0,0.9877055,1.0043356,0.9976136,0.00034463604,0.96330273,4051,537,55337,7,10371 .. -1,10371,0,35880,1
195.02299176985144,-0.6109986068739428,745,301,2,242,557,7,31442,12561,0.0 .. 0.485507,1.0538298 .. 28.077778,1.9977036 .. 34.96221,0.15898278 .. 12.587491,0.3472642 .. 38.362625,0.13822621 .. 7.9084597,38.769444 .. 5.3888917,0.11467902 .. 0.032902088,1.5613878 .. 1.2457927,1.1966538,1.4503974 .. 34.529015,0.022246173,1.2488135 .. 5.2008977,1,0.56580216,0.23863779,1.0,1.0,0.9960191,1.0043356,1.0060105,0.00031904512,0.96330273,4051,519,55337,7,10371 .. -1,10371,0,8843,1
195.07974105207435,0.34898067241979264,745,301,4,243,389,7,30058,1642,1.0 .. 1.0,4.0812035 .. 18.327095,4.580044 .. 30.421139,0.22630303 .. 9.857254,0.616989 .. 26.011078,0.12466159 .. 5.8147774,74.91412 .. 4.703323,0.12867135 .. 0.03691657,1.3627167 .. 1.1680814,1.1853721,1.422432 .. 33.400097,0.024960494,1.0738746 .. 4.7931013,1,0.61221063,0.34315524,1.0,1.0,0.9783097,0.9879532,0.9720055,0.00019141332,0.952,4051,553,55337,7,10371 .. -1,10371,0,13065,2
195.1705738058857,0.3598641895951883,745,301,4,243,722,7,30064,1643,1.0 .. 1.0,11.847727 .. 18.313528,2.9395227 .. 24.89531,0.19240873 .. 13.38045,0.5129004 .. 26.396105,0.16149661 .. 8.317204,22.908083 .. 2.8948202,0.13199425 .. 0.037869927,1.3627167 .. 1.1680814,1.1853721,1.422432 .. 33.400097,0.02560509,1.0738746 .. 4.7931013,1,0.5020603,0.18490975,1.0,1.0,0.9991358,0.9879532,0.9926974,0.00044080438,1.0,4052,515,55600,7,10372 .. -1,10372,0,12492,1
195.2042890842218,0.3472671634337523,745,301,4,243,883,7,30064,1643,1.0 .. 1.0,0.9547552 .. 13.006868,0.94908774 .. 14.829185,0.2857886 .. 7.6804867,0.27861363 .. 15.338097,0.16022317 .. 4.8716736,15.842348 .. 2.865391,0.13823059 .. 0.039659172,1.3627167 .. 1.1680814,1.1853721,1.422432 .. 33.400097,0.026814857,1.0738746 .. 4.7931013,1,0.5591732,0.2325996,1.0,1.0,0.97878534,0.9879532,0.9724781,0.00032992335,1.0,4051,559,55337,7,10371 .. -1,10371,0,2060,1
195.26124844673603,0.2884124780753852,745,301,4,244,506,7,30064,1643,0.0 .. 1.0,0.45335904 .. 16.072401,0.46969956 .. 21.864227,0.15547043 .. 11.060858,0.25580257 .. 14.571057,0.096537545 .. 6.170809,5.5943246 .. 2.905092,0.14573613 .. 0.041812554,1.3353784 .. 1.1939708,1.1853721,1.4178582 .. 33.400097,0.028270828,1.050638 .. 4.899336,1,0.73590684,0.82845473,1.0,1.0,0.97960323,0.98765314,0.9729951,2.0706651e-05,1.0,4051,596,55337,7,10371 .. -1,10371,0,21454,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318.34279202840077,2.646901120835802,7717,301,6,51,982,21,29117,6103,-0.0 .. 0.9698201,2.0141122 .. 58.867867,-0.30240676 .. 79.8288,-0.2535516 .. 25.449942,-0.2924558 .. 69.22061,0.0007514662 .. 15.4802475,0.00034301903 .. 4.3103695,0.32052237 .. 0.09195976,1.2805787 .. 1.229068,1.1913005,1.8439392 .. 20.477808,0.062176984,1.1505401 .. 3.949341,1,0.42227677,0.2997633,1.0,1.

In [4]:
data_boss_good = data_boss_all[overlap_boss_delve]

In [5]:
len(data_boss_good)

944762

In [6]:
px_overlap = np.unique(pxnums_boss[overlap_boss_delve])

In [7]:
RAdelve, DECdelve = hp.pix2ang(32, px_overlap, lonlat=True)

In [8]:
mask1 = np.array(RAdelve)<180
mask2 = np.array(RAdelve)>120
mask3 = np.array(RAdelve)>-180
mask4 = np.array(RAdelve)<-120

mask5 = np.array(DECdelve)>-10
mask6 = np.array(DECdelve)<10
mask12 = mask1*mask2
mask34 = mask3*mask4
mask56 = mask5*mask6
mask1234 = (mask12+mask34)
mask = mask1234 * mask56

In [9]:
px_overlap_strings = np.char.add(np.zeros_like(np.array(px_overlap[mask])).astype('str'), np.array(px_overlap[mask]).astype('str'))

In [ ]:
#running bpz on all pixels
for pixel in px_overlap_strings:
    
    #changing pars file to have correctly labeled output
    list_lines = ['COLUMNS     /data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example.columns\n', f'OUTPUT\t    /data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example_output_{pixel}_no_wise.bpz.h5\n', 'SPECTRA     CWWSB4.list\n', 'PRIOR\t    hdfn_gen\n', 'DZ          0.01\n', 'ZMIN        0.005\n', 'ZMAX        3.505\n', 'MAG         yes\n', 'NEW_AB      no\n', 'MADAU\t    no #TURN OFF MADAU!!!!\n', 'EXCLUDE     none\n', 'CHECK       yes\n', '#ZC          1.0,2.0\n', '#FC          0.2,0.4\n', 'VERBOSE     no\n', '#INTERP      0\n', 'ODDS        0.68\n', 'PROBS       no\n', 'PROBS_LITE  DELVEdata_09602_probs.h5\n', 'GET_Z       yes\n', 'INTERACTIVE yes\n', 'PLOTS       no\n', '#ONLY_TYPE   yes\n']
    pars = open('/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example.pars', mode='w')
    pars.writelines(list_lines)
    pars.close()
    
    #reading delve data
    tab_alex = astropy.table.Table.read(f'/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat_hpx_{pixel}.fits')
    
    #making sure we are only dealing with galaxies
    bandlist = ['G', 'R', 'I', 'Z']
    for band in bandlist:
        tab_alex[f'EXTENDED_CLASS_{band}'] = 1*((tab_alex[f'SPREAD_MODEL_{band}'] + 3*tab_alex[f'SPREADERR_MODEL_{band}']) > 0.005) + 1*((tab_alex[f'SPREAD_MODEL_{band}'] + tab_alex[f'SPREADERR_MODEL_{band}']) > 0.003) + 1*((tab_alex[f'SPREAD_MODEL_{band}'] - tab_alex[f'SPREADERR_MODEL_{band}']) > 0.003)

    tab_alex_f = tab_alex[tab_alex['EXTENDED_CLASS_G']>1]

    #creating the table to be run through bpz
    runthroughbpz = tab_alex_f['QUICK_OBJECT_ID', 'MAG_AUTO_G', 'MAG_AUTO_R', 'MAG_AUTO_I', 'MAG_AUTO_Z', 'MAGERR_AUTO_G', 'MAGERR_AUTO_R', 'MAGERR_AUTO_I', 'MAGERR_AUTO_Z'] #[tab_all['extended_class']>1]
    
    #converting to pandas dataframe
    pdbpz = pd.DataFrame(data=runthroughbpz['QUICK_OBJECT_ID'], columns = ['QUICK_OBJECT_ID'])
    for label in runthroughbpz.columns:
        pdbpz[label] = runthroughbpz[label]
    
    #creating hdf5 file from pandas dataframe, which will be the file run through bpz
    pdbpz.to_hdf(f'/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_{pixel}_no_wise.h5', key='df')
    
    #changing file name for specific pixel
    catalog_name = f'/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_{pixel}_no_wise.h5'
    
    #running bpz using subprocess
    sp.run(['python', '/data/des90.a/data/raulteix/software/DESC_BPZ/scripts/bpz.py', catalog_name, '-P', '/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example.pars'])

In [ ]:
#getting bpz redshifts
data = astropy.table.Table()
for pixel in px_overlap_strings:
    tab_alex = astropy.table.Table.read(f'/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat_hpx_{pixel}.fits')

    bpzdata = h5py.File(f'/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example_output_{pixel}_no_wise.bpz.h5')
    bpzdatapd = pd.DataFrame()
    for key in list(bpzdata.keys()):
        bpzdatapd[key]=list(bpzdata[key])

    bpzdatatable = astropy.table.Table.from_pandas(bpzdatapd)
    bpzdatatable['ID'].name = 'QUICK_OBJECT_ID'
    datai = astropy.table.join(bpzdatatable, tab_alex_f, keys='QUICK_OBJECT_ID')
    data = astropy.table.vstack(data, datai)
    
len(data)

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
boss_cd = SkyCoord(ra=data_boss_good['RA']*u.degree, dec=data_boss_good['DEC']*u.degree)
bpz_cd = SkyCoord(ra=data['RA']*u.deg, dec=data['DEC']*u.deg)
idx, d2d, d3d = boss_cd.match_to_catalog_sky(bpz_cd)

In [ ]:
good_matches = d2d < 1.0*u.arcsec
print(len(np.unique(idx[good_matches])), np.count_nonzero(good_matches))

In [ ]:
idx_, d2d_, d3d_ = bpz_cd.match_to_catalog_sky(boss_cd)

In [ ]:
good_matches_ = d2d_ < 1.0*u.arcsec
print(len(np.unique(idx_[good_matches_])), np.count_nonzero(good_matches_))

In [ ]:
data_all = astropy.table.hstack([data_boss_good[good_matches], data[idx][good_matches]])

In [ ]:
data_all.write('/data/des90.a/data/raulteix/data/DELVE-BOSS.fits', format='fits')